## 01 Introduction and Setup
## 02 V8 The Javascript Engine
## 03 The Node Core

## 04 Modules Exports and Require
### 027 exports vs module.exports
require() 实际上是类似下面的 wrapper 函数。其中第一个参数其实就是 module.exports。
```javascript
fn(module.exports, require, module, __filename, __dirname);
```

```javascript
(function (exports, require, module, __filename, __dirname) {

    var greet = function() {
        console.log('Hello!');
    };

    module.exports = greet;
});
```

In [1]:
// 模拟 exports 换成其它值
(function() {

    var module = {};
    module.exports = {};
    var require = function() {};

    // 模拟 require('./greet.js')
    (function (exports, require, module, __filename, __dirname) {

        // 这里相当于是 greet.js
        exports = function() {
            console.log('Hello!');
        };

        console.log(exports);
        console.log(module.exports);

    })(module.exports, require, module, 'filename', 'dirname');
    return module.exports; // simulate return in require()
})();

上面例子让 exports 指向了不同的对象。**一般可能期望 exports 改了，module.exports 也跟着修改，但这里不是这样。**

In [2]:
// 再模拟，改变 exports 内部（mutate exports），但 exports 本身引用不变
(function() {

    var module = {};
    module.exports = {};
    var require = function() {};

    // 模拟 require('./greet.js')
    (function (exports, require, module, __filename, __dirname) {

        // 这里相当于是 greet.js
        // NOTE: 由于某些原因，函数并不会输出，所以又加了字符串属性来验证
        exports.greet = function() { console.log('Hello!'); };
        exports.greetStr = `function() { console.log('Hello!'); }`;

        console.log(exports);
        console.log(module.exports);

    })(module.exports, require, module, 'filename', 'dirname');
    return module.exports; // simulate return in require()
})();

这里容易造成误解，所以简单的方法：***Just use module.exports**

### 028 Requiring Native Core Modules

In [3]:
var util = require('util');

var name = 'Tony';
var greeting = util.format('Hello, %s', name);
//util.log(greeting); // jupyter 里面似乎不工作
console.log(greeting);

如果自己也有同名的 module 怎么办？问题不大，自己的使用相对路径：
```javascript
require('./util');
```
但是还是不建议使用相同的名字，造成混淆。

### 029 Modules and ES6

- greet.js
```javascript
export function greet() {
    console.log('Hello');
}
```
- app.js
```javascript
import * as greetr from 'greet';
greetr.greet();
```

## 05 Events and the Event Emitter

### 031 Conceptual Aside Events
- System Events
  - C++ Core (libuv)
- Custom Event
  - Javascript Core (Event Emitter)

### 033 The Node Event Emitter - Part 1
下面是一个模拟的 Event Emitter

In [4]:
(function() {
    var module = {};
    var require = function() {
        return module.exports;
    };

    // ----- begin of emitter.js -----
    function Emitter() {
        this.events = {};
    }

    Emitter.prototype.on = function(type, listener) {
        this.events[type] = this.events[type] || [];
        this.events[type].push(listener);
    };

    Emitter.prototype.emit = function(type) {
        if (this.events[type]) {
            this.events[type].forEach(function(listener) {
                listener();
            });
        }
    }
    
    module.exports = Emitter;
    // ----- end of emitter.js -----

    // ----- begin of app.js -----
    var Emitter = require('./emitter');
    var emtr = new Emitter();

    emtr.on('greet', function() {
        console.log('Somewhere, some said hello.');
    });

    emtr.on('greet', function() {
        console.log('A greeting occurred!');
    });

    console.log('Hello!');
    emtr.emit('greet');
})();

### 034 The Node Event Emitter - Part 2
Node 提供的 Event Emitter

In [5]:
(function() {
    var Emitter = require('events');
    var emtr = new Emitter();

    emtr.on('greet', function() {
        console.log('Somewhere, some said hello.');
    });

    emtr.on('greet', function() {
        console.log('A greeting occurred!');
    });

    console.log('Hello!');
    emtr.emit('greet');
})();

Hello!
Somewhere, some said hello.
A greeting occurred!


怎么避免使用这些 magic strings?

In [6]:
(function() {
    var module = {};
    var _require = require;
    require = function(name) {
        return (name === './config') ? module.exports : _require(name);
    };

    // ----- begin of config.js -----
    module.exports = {
        events: {
            GREET: 'greet'
        }
    };

    // ----- begin of app.js -----
    var Emitter = require('events');
    var eventConfig = require('./config').events;
    var emtr = new Emitter();

    emtr.on(eventConfig.GREET, function() {
        console.log('Somewhere, some said hello.');
    });

    emtr.on(eventConfig.GREET, function() {
        console.log('A greeting occurred!');
    });

    console.log('Hello!');
    emtr.emit(eventConfig.GREET);
})();

Hello!
Somewhere, some said hello.
A greeting occurred!


### 036 Inheriting From the Event Emitter

In [7]:
(function() {

    var EventEmitter = require('events');
    var util = require('util');
    
    function Greetr() {
        this.greeting = 'Hello world!';
    }

    util.inherits(Greetr, EventEmitter);

    Greetr.prototype.greet = function() {
        console.log(this.greeting);
        this.emit('greet');
    };

    var greeter1 = new Greetr();
    greeter1.on('greet', function() {
        console.log('Someone greeted!')
    });

    greeter1.greet();
})();

Hello world!
Someone greeted!


如果想要给 event 参数？

In [8]:
(function() {

    var EventEmitter = require('events');
    var util = require('util');
    
    function Greetr() {
        this.greeting = 'Hello world!';
    }

    util.inherits(Greetr, EventEmitter);

    Greetr.prototype.greet = function(data) {
        console.log(this.greeting + ': ' + data);
        this.emit('greet', data);
    };

    var greeter1 = new Greetr();
    greeter1.on('greet', function(data) {
        console.log('Someone greeted: ' + data)
    });

    greeter1.greet('Tony');
})();

Hello world!: Tony
Someone greeted: Tony


### 037 Javascript Aside Node ES6 and Template Literals
如果使用 ES6，需要文件 jsconfig.json
```json
    {
        "compilerOptions": {
            "target": "ES6"
        }
    }
```

对于浏览器，Template Literals不一定能用，但是 node 很容易确定ES6可用

In [9]:
(function() {
    var name = 'John Doe';

    var greet = 'Hello ' + name;
    var greet2 = `Hello ${ name }`; // Template Literals
    
    console.log(greet);
    console.log(greet2);
})();

Hello John Doe
Hello John Doe


### 038 Javascript Aside .call and .apply

In [10]:
(function() {
    var obj = {
        name: 'John Doe',
        greet: function() {
            console.log(`Hello ${ this.name }`);
        }
    };

    obj.greet();
    obj.greet.call({ name: 'Jane Doe' });
    obj.greet.apply({ name: 'Jane Doe' });
})();

Hello John Doe
Hello Jane Doe
Hello Jane Doe


### 039 Inheriting From the Event Emitter - Part 2

In [11]:
(function() {

    var EventEmitter = require('events');
    var util = require('util');
    
    function Greetr() {
        EventEmitter.call(this); // 和前面的例子，区别在这里
        this.greeting = 'Hello world!';
    }

    util.inherits(Greetr, EventEmitter);

    Greetr.prototype.greet = function(data) {
        console.log(this.greeting + ': ' + data);
        this.emit('greet', data);
    };

    var greeter1 = new Greetr();
    greeter1.on('greet', function(data) {
        console.log('Someone greeted: ' + data)
    });

    greeter1.greet('Tony');
})();

Hello world!: Tony
Someone greeted: Tony
